In [1]:
import numpy as np
import zipfile
from datasets import Dataset
import pandas as pd

import os
# run as much as possible accelerated by apple silicon, fall back to cpu if not possible
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch

import sys
sys.path.append('..')
from scripts.audio import audio_file_to_array
from scripts.ipa import timit2ipa


/Users/alex/Desktop/CS/Startups/Koel/ML/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set espeak library path for macOS
if sys.platform == "darwin":
    from phonemizer.backend.espeak.wrapper import EspeakWrapper

    _ESPEAK_LIBRARY = "/opt/homebrew/Cellar/espeak/1.48.04_1/lib/libespeak.1.1.48.dylib"
    EspeakWrapper.set_library(_ESPEAK_LIBRARY)

In [3]:
from transformers import AutoProcessor, AutoModelForCTC, AutoTokenizer, AutoFeatureExtractor

In [4]:
OUTPUT_DIR = os.path.join('..', 'models', 'timit-xlsr-finetune')
PRE_TRAINED_ID = "facebook/wav2vec2-lv-60-espeak-cv-ft"

# Load Data

In [5]:
timit = zipfile.ZipFile('../.data/TIMIT.zip', 'r')
timit_files = timit.namelist()

In [6]:
training_files = list(set(map(lambda x: x.split('.')[0], filter(lambda x: x.startswith('data/TRAIN'), timit_files))))
test_files = list(set(map(lambda x: x.split('.')[0], filter(lambda x: x.startswith('data/TEST'), timit_files))))

In [7]:
def timit_file_to_dict(filename):
    with timit.open(filename + '.PHN') as phn_file:
        phonemes = []
        for line in phn_file.read().decode('utf-8').split('\n'):
            if line == '':
                continue
            _, _, phoneme = line.split()
            phonemes.append(timit2ipa(phoneme))
    return {'phonemes': phonemes, 'wav_filename': filename + '.WAV'}

In [8]:
WAV_HEADER_SIZE = 44
def zipped_wav_to_array(filename):
    with timit.open(filename) as wav_file:
        return np.frombuffer(wav_file.read(), dtype=np.int16)[WAV_HEADER_SIZE//2:]

In [9]:
def files_to_df(files):
    records = []
    for filename in files:
        parsed = timit_file_to_dict(filename)
        parsed['audio'] = zipped_wav_to_array(parsed['wav_filename'])
        del parsed['wav_filename']
        parsed['ipa'] = "".join(parsed['phonemes'])
        del parsed['phonemes']
        records.append(parsed)
    return pd.DataFrame(records)

In [10]:
train_df = files_to_df(training_files)
train_df.head()

,audio,ipa
0,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",oʊnʔæsmitɨkɛɹiʔɨnʔɔɪliɹæglaɪkðæt
1,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ɹiɾiŋʔɨnpɔɹlaɪtɪvʒʉʔaɪstɹeɪn
2,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ɨmplɔɪɨʔleɪɔfskoʊɨntsaɪdɪdwɨθɨkʌmpn̩izɹɨɔɹgɨnɨ...
3,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ægaɪpsɨnbɑŋgoʊzɚmjʉzɨkl̩ ɨntstə̥mɨnts
4,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ʃiɦæddʒɚdɑɹksʉɾɨngɹisiwɔɹʃwɔɾɚɔljɪɚ


In [11]:
test_df = files_to_df(test_files)
test_df.head()

,audio,ipa
0,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ælsiɪmeɪksboʊnzɨntiθstɹɔŋ
1,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",itsɚɹiʌzɑɹkɨnvinjɛʔfɔɹeɪkwɪklʌnttʃ
2,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ʃiɦædjɔɹdɑɹksʉtɨngɹisiwɔʃwɔɾɚʔɔljɪɚ
3,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ɪfkɛɹl̩ kʌmztəmɑɹəɦævɝɹeɪndʒfɝmiɾiŋɨʔtʉ
4,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",hiɪdmɪɾɪɦɪzbiɦeɪvjɚɨndɪfɛndɪɾɪt


In [12]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# Extend Phoneme Vocab

In [13]:
timit_vocab = set("".join(train_df['ipa'])) | set("".join(test_df['ipa']))

In [14]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_ID)
vocab = tokenizer.get_vocab()

In [15]:
additional_vocab = timit_vocab.difference(set(vocab.keys()) | {' '})
tokenizer.add_tokens(list(additional_vocab))
print(additional_vocab)

{'̥', 'ɝ', '̃', '̩', 'g', 'ɦ', '̍'}


In [16]:
# replace ' ' in ipa with tokenizer.pad_token
train_ds = train_ds.map(lambda x: {'audio': x['audio'], 'ipa': x['ipa'].replace(' ', tokenizer.pad_token)})
test_ds = test_ds.map(lambda x: {'audio': x['audio'], 'ipa': x['ipa'].replace(' ', tokenizer.pad_token)})

Map: 100%|██████████| 1680/1680 [00:24<00:00, 69.78 examples/s]


# Update Feature Extraction

In [17]:
feature_extractor = AutoFeatureExtractor.from_pretrained(PRE_TRAINED_ID)

We don't need to update the feature extractor since it has been pretrained on 16kHz audio which matches the TIMIT dataset.

For datasets with different sampling rates, the feature extractor should be updated or the audio resampled (easier).

This is also where code to add extra features (such as conditioning on speaker's native language etc.) would be added.

# Save Updated Tokenizer and Feature Extractor

In [18]:
processor = AutoProcessor.from_pretrained(PRE_TRAINED_ID)
processor.tokenizer = tokenizer
processor.feature_extractor = feature_extractor
processor.save_pretrained(OUTPUT_DIR)

[]

# Preprocess Data

In [19]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["audio"], sampling_rate=feature_extractor.sampling_rate).input_values
    batch["labels"] = processor(text=batch["ipa"]).input_ids
    return batch

In [20]:
train_ds_prepared = train_ds.map(prepare_dataset, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4): 100%|██████████| 4620/4620 [01:06<00:00, 69.69 examples/s] 


In [21]:
test_ds_prepared = test_ds.map(prepare_dataset, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4): 100%|██████████| 1680/1680 [00:27<00:00, 60.24 examples/s] 


In [22]:
train_ds_prepared.to_pandas().head()

,audio,ipa,input_values,labels
0,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",oʊnʔæsmitɨkɛɹiʔɨnʔɔɪliɹæglaɪkðæt,"[18.135152024062002, 21.17466781012437, 20.460...","[49, 29, 18, 5, 17, 8, 41, 4, 14, 4, 35, 8, 41..."
1,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ɹiɾiŋʔɨnpɔɹlaɪtɪvʒʉʔaɪstɹeɪn,"[18.094273809159557, 21.125626761823945, 20.41...","[6, 63, 4, 12, 41, 27, 37, 68, 15, 36, 18, 37,..."
2,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ɨmplɔɪɨʔleɪɔfskoʊɨntsaɪdɪdwɨθɨkʌmpn̩izɹɨɔɹgɨnɨ...,"[16.754583715965623, 19.561161243340255, 18.90...","[37, 26, 68, 14, 13, 18, 30, 14, 8, 49, 18, 7,..."
3,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ægaɪpsɨnbɑŋgoʊzɚmjʉzɨkl̩<pad>ɨntstə̥mɨnts,"[15.056215957312128, 17.58146924092826, 16.988...","[8, 17, 35, 20, 66, 7, 44, 30, 396, 20, 5, 13,..."
4,"[25971, 30303, 29285, 26995, 28271, 11552, 131...",ʃiɦæddʒɚdɑɹksʉɾɨngɹisiwɔɹʃwɔɾɚɔljɪɚ,"[19.301096442287164, 22.5337553816438, 21.7740...","[14, 38, 37, 397, 8, 17, 35, 20, 66, 7, 44, 30..."


# Fine-Tune Model

In [23]:
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    def __init__(self, processor: AutoProcessor, padding=True, max_length=None, max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None):
        self.processor = processor
        self.padding = padding
        self.max_length = max_length
        self.max_length_labels = max_length_labels
        self.pad_to_multiple_of = pad_to_multiple_of
        self.pad_to_multiple_of_labels = pad_to_multiple_of_labels

    def __call__(self, features: "list[dict[str, list[int] | torch.Tensor]]") -> "dict[str, torch.Tensor]":
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [24]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [25]:
def cer(pred, label):
    distances = np.zeros((len(pred) + 1, len(label) + 1))

    for t1 in range(len(pred) + 1):
        distances[t1][0] = t1

    for t2 in range(len(label) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(pred) + 1):
        for t2 in range(1, len(label) + 1):
            if (pred[t1-1] == label[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(pred)][len(label)] / len(label)

In [26]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False) # labels are already grouped as they should be

    err = cer(pred_str, label_str)

    return {"cer": err}

In [27]:
# testing
label_str = 'aab'
pred_str = 'aaac'
print(cer(pred_str, label_str))

0.6666666666666666


In [28]:
model = AutoModelForCTC.from_pretrained(
    PRE_TRAINED_ID, 
    pad_token_id=processor.tokenizer.pad_token_id, 
    gradient_checkpointing=True,
    ctc_zero_infinity = True
)

In [29]:
# model.resize_token_embeddings(len(tokenizer))
old_weights = model.lm_head.weight.t().detach()
old_bias = model.lm_head.bias.detach()
new_layer = torch.nn.Linear(model.lm_head.in_features, len(processor.tokenizer))
new_layer.weight.data[:model.lm_head.out_features, :] = old_weights.t()
new_layer.bias.data[:model.lm_head.out_features] = old_bias
model.lm_head = new_layer
model.config.vocab_size = len(processor.tokenizer)

In [30]:
model.freeze_feature_encoder()

In [31]:
# sanity checks
max_class_id = 0
for label_sequence in train_ds_prepared['labels']:
    for class_id in label_sequence:
        max_class_id = max(max_class_id, class_id)
assert max_class_id == model.config.vocab_size - 1

for label_sequence in train_ds_prepared['labels']:
    for class_id in label_sequence:
        assert class_id >= 0 and class_id <  model.config.vocab_size

for label_sequence in train_ds_prepared['labels']:
    assert  len(label_sequence) < 1024
    assert len(label_sequence) >= 5

In [36]:
from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir=OUTPUT_DIR,
#     group_by_length=True,
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=2,
#     eval_strategy="steps",
#     num_train_epochs=50,
#     fp16=False, # set to true when training with CUDA?
#     save_steps=500,
#     eval_steps=500,
#     logging_steps=500,
#     learning_rate=3e-4,
#     gradient_checkpointing=True,
#     warmup_steps=1000,
#     save_total_limit=2,
#     load_best_model_at_end=True,
# )

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    group_by_length=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    eval_strategy="steps",
    num_train_epochs=1,
    fp16=False,
    save_steps=100,
    eval_steps=100,
    logging_steps=100,
    learning_rate=3e-4,
    warmup_steps=0,
    save_total_limit=3,
    gradient_checkpointing=True,
    load_best_model_at_end=True,
    max_grad_norm=1.0,
)

In [37]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds_prepared.select(range(100)),
    # train_dataset=train_ds_prepared,
    eval_dataset=test_ds_prepared.select(range(100)),
    # eval_dataset=test_ds_prepared,
    processing_class=processor.feature_extractor,
)

In [38]:
torch.autograd.set_detect_anomaly(True) # debug NaNs, disable for real training to improve performance
trainer.train() # resume_from_checkpoint=True to resume training if training was interrupted

  0%|          | 0/7 [00:00<?, ?it/s]/Users/alex/Desktop/CS/Startups/Koel/ML/venv/lib/python3.8/site-packages/torch/nn/functional.py:2705: UserWarning: The operator 'aten::_ctc_loss' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch.ctc_loss(
/Users/alex/Desktop/CS/Startups/Koel/ML/venv/lib/python3.8/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
                                             
100%|██████████| 7/7 [01:08<00:00,  9.82s/it]

{'train_runtime': 68.7438, 'train_samples_per_second': 1.455, 'train_steps_per_second': 0.102, 'train_loss': 7413.9765625, 'epoch': 1.0}


TrainOutput(global_step=7, training_loss=7413.9765625, metrics={'train_runtime': 68.7438, 'train_samples_per_second': 1.455, 'train_steps_per_second': 0.102, 'total_flos': 1.5305473943388e+16, 'train_loss': 7413.9765625, 'epoch': 1.0})

# Evaluate

In [41]:
trainer.evaluate()

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/Users/alex/Desktop/CS/Startups/Koel/ML/venv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
100%|██████████| 13/13 [00:17<00:00,  1.34s/it]


{'eval_loss': 3396.741943359375,
 'eval_cer': 1.0,
 'eval_runtime': 26.6438,
 'eval_samples_per_second': 3.753,
 'eval_steps_per_second': 0.488,
 'epoch': 1.0}

In [42]:
old_model = AutoModelForCTC.from_pretrained(PRE_TRAINED_ID).to(model.device)

In [43]:
finetuned_model = AutoModelForCTC.from_pretrained(os.path.join(OUTPUT_DIR, 'checkpoint-7'))

In [44]:
# check for nan weights and replace with old values if any
for name, param in finetuned_model.named_parameters():
    if torch.isnan(param).any():
        print(name)
        param.data = old_model.state_dict()[name].data

In [46]:
# compare weights
is_diff = False
for name, param in finetuned_model.named_parameters():
    if not torch.equal(param, old_model.state_dict()[name].to(finetuned_model.device)):
        is_diff = True
        break
print("Weights are different, training did something" if is_diff else "Weights are the same, training did nothing")

Weights are different, training did something


In [47]:
def predict(model, wav_file):
    speech = audio_file_to_array(wav_file)
    input_values = processor(speech, sampling_rate=16000, return_tensors="pt").input_values.type(torch.float32).to(model.device)

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    return processor.decode(predicted_ids[0])
    

In [48]:
print('Checkpoint Transcription:', predict(finetuned_model, os.path.join('..', 'data', 'alexIsConfused.wav')))

Checkpoint Transcription: h ɛ l ɛ l ɛ s ɛ z ɛ k t ɹ k ɛ n f iː


In [49]:
print('Best Fine-Tune Transcription:', predict(trainer.model, os.path.join('..', 'data', 'alexIsConfused.wav')))

Best Fine-Tune Transcription: h ɛ l ɛ l ɛ s ɛ z ɛ k t ɹ k ɛ n f iː


In [50]:
print('Old Transcription:', predict(old_model, os.path.join('..', 'data', 'alexIsConfused.wav')))

Old Transcription: h ɛ l oʊ æ l ɪ k s ɪ z ɛ k s t ɾ ɹ ə k ə n f iː l ʊ s t
